In [2]:
import os
import math
import operator
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from textblob import TextBlob
from gsdmm import MovieGroupProcess

## pre-processed text

In [3]:
# data = pd.read_pickle("../../../sampled_processed_extant_posts.pkl")
data = pd.read_pickle("../../../sampled_processed_extant_posts.pkl")

In [4]:
print(f"rows in data: {len(data)}")

rows in data: 2695354


In [5]:
data.head()

,text_clean_space,text_clean_punc_lower,len_clean,tokens,tokens_clean,tokens_lemma,bigrams,trigrams
1021188,"Hmmm. Interesting. See, not so sophisticated hah.",hmmm interesting see not so sophisticated hah,45,"[hmmm, interesting, see, not, so, sophisticate...","[hmmm, interesting, see, sophisticated, hah]","[hmmm, interesting, see, sophisticated, hah]","[(hmmm, interesting), (interesting, see), (see...","[(hmmm, interesting, see), (interesting, see, ..."
9531089,Thank you!!! I love you!!!,thank you i love you,20,"[thank, you, i, love, you]","[thank, love]","[thank, love]","[(thank, love)]",[]
2720963,You do realise you're contradicting yourself r...,you do realise youre contradicting yourself right,49,"[you, do, realise, youre, contradicting, yours...","[realise, contradicting, right]","[realise, contradicting, right]","[(realise, contradicting), (contradicting, rig...","[(realise, contradicting, right)]"
9086559,"Per the below comments, it will vary. That sai...",per the below comments it will vary that said ...,164,"[per, the, below, comments, it, will, vary, th...","[per, comments, vary, said, would, expect, cou...","[per, comment, vary, said, would, expect, coul...","[(per, comment), (comment, vary), (vary, said)...","[(per, comment, vary), (comment, vary, said), ..."
9102373,,,0,[],[],[],[],[]


## full data with dates

In [7]:
posts_pkl_path = os.path.join('..', '..', '..', "combined_bigquery_processed.pkl")
posts_df = pd.read_pickle(posts_pkl_path)

#### merge data

In [8]:
data = data.merge(posts_df, left_index = True, right_index = True, how = 'left')

In [9]:
data['date'] = pd.to_datetime(data['created_date'], format='%Y-%m-%d')
data['date'] = data['date'].dt.date

#### subset to august date range

In [10]:
subset_date2 = data[(data['date']>= datetime.strptime('2019-08-09', '%Y-%m-%d').date())
     & (data['date']<= datetime.strptime('2019-08-11', '%Y-%m-%d').date())]

In [11]:
print(subset_date2.shape)
subset_date2.head()

(22823, 31)


,text_clean_space,text_clean_punc_lower,len_clean,tokens,tokens_clean,tokens_lemma,bigrams,trigrams,selftext,score_hidden,...,controversiality,gilded,id,subreddit,ups,distinguished,author_flair_css_class,created_date,post_type,date
8368084,SPARKLIN TATERS!!,sparklin taters,15,"[sparklin, taters]","[sparklin, taters]","[sparklin, tater]","[(sparklin, tater)]",[],SPARKLIN TATERS!!,NaN,...,0,0,ewgiz57,The_Donald,NaN,NaN,army,2019-08-09,extant,2019-08-09
7622971,*puts on Tshirt* [LOOKIT ME! IM A FREEDOM FIGH...,puts on tshirt lookit me im a freedom fighterh...,70,"[puts, on, tshirt, lookit, me, im, a, freedom,...","[puts, tshirt, lookit, im, freedom, fighterhtt...","[put, tshirt, lookit, im, freedom, fighterhttp...","[(put, tshirt), (tshirt, lookit), (lookit, im)...","[(put, tshirt, lookit), (tshirt, lookit, im), ...",*puts on Tshirt*\n\n[LOOKIT ME! IM A FREEDOM F...,NaN,...,0,0,ewmjnzo,The_Donald,NaN,NaN,england,2019-08-11,extant,2019-08-11
8319493,vegans get the fuck outta here with that soy s...,vegans get the fuck outta here with that soy s...,131,"[vegans, get, the, fuck, outta, here, with, th...","[vegans, get, fuck, outta, soy, shit, yo, mayb...","[vegan, get, fuck, outta, soy, shit, yo, maybe...","[(vegan, get), (get, fuck), (fuck, outta), (ou...","[(vegan, get, fuck), (get, fuck, outta), (fuck...",vegans\n\nget the fuck outta here with that so...,NaN,...,0,0,ewl5c9w,The_Donald,NaN,NaN,russia,2019-08-11,extant,2019-08-11
7872118,We must all post this on every media available...,we must all post this on every media available...,161,"[we, must, all, post, this, on, every, media, ...","[must, post, every, media, available, great, r...","[must, post, every, medium, available, great, ...","[(must, post), (post, every), (every, medium),...","[(must, post, every), (post, every, medium), (...",We must all post this on every media available...,NaN,...,0,0,ewkgz23,The_Donald,NaN,NaN,NaN,2019-08-10,extant,2019-08-10
8439541,So gafftastically funny. . Keep him in this race!,so gafftastically funny keep him in this race,46,"[so, gafftastically, funny, keep, him, in, thi...","[gafftastically, funny, keep, race]","[gafftastically, funny, keep, race]","[(gafftastically, funny), (funny, keep), (keep...","[(gafftastically, funny, keep), (funny, keep, ...",So gafftastically funny. ����. Keep him in th...,NaN,...,0,0,ewlgqw6,The_Donald,NaN,NaN,MURICA,2019-08-11,extant,2019-08-11


In [12]:
del data

In [26]:
subset_date2 = subset_date2 [subset_date2['author']!='AutoModerator']
subset_date2.shape

(20600, 31)

# short text lda model

### review vocab for date range

In [132]:
vocab_counts = {}
token_length = {}
sample_data = subset_date2['bigrams']
for token_list in sample_data:
    for token in token_list:
        token_length[token] = len(token)
        vocab_counts[token] = vocab_counts.get(token, 0) + 1

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [133]:
sorted_vocab_counts = dict( sorted(vocab_counts.items(), key=operator.itemgetter(1),reverse=True))
# print('Dictionary in descending order by value : ', sorted_vocab_counts)
# like, would, see, get , im, thats, may, got, 

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [134]:
sorted_vocab_len = dict( sorted(token_length.items(), key=operator.itemgetter(1),reverse=True))
# print('Dictionary in descending order by value : ', sorted_vocab_len)

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [135]:
sorted_vocab_counts

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{('suicide', 'watch'): 244,
 ('look', 'like'): 204,
 ('gon', 'na'): 162,
 ('red', 'flag'): 151,
 ('im', 'sure'): 135,
 ('deep', 'state'): 112,
 ('flag', 'law'): 99,
 ('body', 'count'): 89,
 ('white', 'supremacist'): 87,
 ('gun', 'control'): 84,
 ('year', 'ago'): 83,
 ('sound', 'like'): 81,
 ('fake', 'news'): 79,
 ('conspiracy', 'theory'): 78,
 ('social', 'medium'): 75,
 ('president', 'trump'): 72,
 ('got', 'ta'): 71,
 ('mass', 'shooting'): 70,
 ('make', 'sure'): 69,
 ('bill', 'clinton'): 68,
 ('united', 'state'): 68,
 ('many', 'people'): 67,
 ('donald', 'trump'): 67,
 ('white', 'people'): 65,
 ('year', 'old'): 59,
 ('background', 'check'): 57,
 ('make', 'sense'): 56,
 ('people', 'like'): 55,
 ('even', 'though'): 55,
 ('trump', 'supporter'): 55,
 ('open', 'carry'): 53,
 ('jeffrey', 'epstein'): 50,
 ('he', 'dead'): 49,
 ('clinton', 'body'): 48,
 ('every', 'time'): 47,
 ('something', 'like'): 47,
 ('new', 'york'): 46,
 ('witness', 'protection'): 45,
 ('pretty', 'much'): 45,
 ('feel', 'lik

In [31]:
# vocab_data = [[k, v, vocab_counts[k]] for k, v in sorted_vocab_len.items()]
# vocab_data[0:10]

In [10]:
# import csv
# def write_list_to_csv(lines, file_name):
#     """
#     write list to csv
#     :param lines: list of data to write to csv
#     :param file_name: output file name
#     """
#     print(f"writine file to {file_name}")
#     with open(file_name, 'w') as writeFile:
#         writer = csv.writer(writeFile)
#         writer.writerows(lines)

In [11]:
# write_list_to_csv(vocab_data, "vocab_data_sample.csv")

In [45]:
vocab_size = len(vocab_counts)
print(f"vocab size {vocab_size}")

vocab size 180024


## train topic model

In [58]:
# Init of the Gibbs Sampling Dirichlet Mixture Model algorithm
mgp = MovieGroupProcess(K=5, alpha=0.1, beta=0.8, n_iters=50)

In [59]:
fit1 = mgp.fit(sample_data , vocab_size)

In stage 0: transferred 15798 clusters with 5 clusters populated
In stage 1: transferred 13850 clusters with 5 clusters populated
In stage 2: transferred 13048 clusters with 5 clusters populated
In stage 3: transferred 12580 clusters with 5 clusters populated
In stage 4: transferred 12394 clusters with 5 clusters populated
In stage 5: transferred 12310 clusters with 5 clusters populated
In stage 6: transferred 12226 clusters with 5 clusters populated
In stage 7: transferred 12115 clusters with 5 clusters populated
In stage 8: transferred 12168 clusters with 5 clusters populated
In stage 9: transferred 11959 clusters with 5 clusters populated
In stage 10: transferred 12097 clusters with 5 clusters populated
In stage 11: transferred 11959 clusters with 5 clusters populated
In stage 12: transferred 11871 clusters with 5 clusters populated
In stage 13: transferred 11877 clusters with 5 clusters populated
In stage 14: transferred 11817 clusters with 5 clusters populated
In stage 15: transfe

In [54]:
doc_count = np.array(mgp.cluster_doc_count)
print('Number of documents per topic :', doc_count)

Number of documents per topic : [5078 7633 5264 2625]


In [55]:
# Topics sorted by the number of document they are allocated to
top_index = doc_count.argsort()[-7:][::-1]
print('Most important clusters (by number of docs inside):', top_index)

Most important clusters (by number of docs inside): [1 2 0 3]


In [56]:
def top_words(cluster_word_distribution, top_cluster, values):
    for cluster in top_cluster:
        sort_dicts =sorted(mgp.cluster_word_distribution[cluster].items(), key=lambda k: k[1], reverse=True)[:values]
        print('Cluster %s : %s'%(cluster,sort_dicts))
        print(' — — — — — — — — — ')

In [57]:
top_words(mgp.cluster_word_distribution, top_index, 20)

Cluster 1 : [(('look', 'like'), 156), (('gon', 'na'), 142), (('body', 'count'), 88), (('im', 'sure'), 75), (('deep', 'state'), 66), (('sound', 'like'), 65), (('conspiracy', 'theory'), 64), (('bill', 'clinton'), 62), (('fake', 'news'), 59), (('many', 'people'), 48), (('clinton', 'body'), 47), (('witness', 'protection'), 45), (('he', 'dead'), 42), (('president', 'trump'), 42), (('social', 'medium'), 41), (('pretty', 'sure'), 36), (('white', 'supremacist'), 35), (('got', 'ta'), 35), (('get', 'away'), 35), (('people', 'like'), 34)]
 — — — — — — — — — 
Cluster 2 : [(('gun', 'control'), 47), (('white', 'people'), 41), (('year', 'ago'), 34), (('point', 'personal'), 34), (('personal', 'privilege'), 34), (('illegal', 'alien'), 34), (('trump', 'train'), 28), (('would', 'take'), 24), (('video', 'game'), 24), (('background', 'check'), 22), (('white', 'supremacist'), 22), (('pretty', 'much'), 22), (('got', 'ta'), 22), (('go', 'back'), 20), (('cant', 'stop'), 20), (('10', 'billion'), 20), (('white',

In [136]:
# mgp.choose_best_label(list(sample_data)[1])
best_labels = sample_data.apply(lambda row: mgp.choose_best_label(row))
best_labels 

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


8368084    (1, 0.3270836866013497)
7622971    (3, 0.9491141135604281)
8319493    (0, 0.9999824398067418)
7872118    (2, 0.9999895989205586)
8439541    (4, 0.4692790064766138)
                    ...           
8098573    (0, 0.0)               
8468503    (3, 1.0)               
8341451    (2, 0.4087461062147859)
8024592    (4, 0.9999944489691726)
8012975    (0, 0.9999999999399108)
Name: bigrams, Length: 20600, dtype: object

In [137]:
sample_df = sample_data.to_frame()
best_labels = best_labels.to_frame()
sample_df = sample_df.merge(best_labels, left_index=True, right_index=True)
sample_df.head()

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,bigrams_x,bigrams_y
8368084,"[(sparklin, tater)]","(1, 0.3270836866013497)"
7622971,"[(put, tshirt), (tshirt, lookit), (lookit, im), (im, freedom), (freedom, fighterhttpsmagaimgnetimg8p67png)]","(3, 0.9491141135604281)"
8319493,"[(vegan, get), (get, fuck), (fuck, outta), (outta, soy), (soy, shit), (shit, yo), (yo, maybe), (maybe, ate), (ate, steak), (steak, eye), (eye, would), (would, point), (point, direction), (direction, booker)]","(0, 0.9999824398067418)"
7872118,"[(must, post), (post, every), (every, medium), (medium, available), (available, great), (great, red), (red, pilling), (pilling, ammo), (ammo, point), (point, moderate), (moderate, lefty), (lefty, critical), (critical, thinking), (thinking, skill), (skill, kick), (kick, agaisnt)]","(2, 0.9999895989205586)"
8439541,"[(gafftastically, funny), (funny, keep), (keep, race)]","(4, 0.4692790064766138)"


In [138]:
sample_df['best_label'] = sample_df['bigrams_y'].apply(lambda row:row[0])
sample_df['best_label_prob'] = sample_df['bigrams_y'].apply(lambda row:row[1])
sample_df.head()

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,bigrams_x,bigrams_y,best_label,best_label_prob
8368084,"[(sparklin, tater)]","(1, 0.3270836866013497)",1,0.327084
7622971,"[(put, tshirt), (tshirt, lookit), (lookit, im), (im, freedom), (freedom, fighterhttpsmagaimgnetimg8p67png)]","(3, 0.9491141135604281)",3,0.949114
8319493,"[(vegan, get), (get, fuck), (fuck, outta), (outta, soy), (soy, shit), (shit, yo), (yo, maybe), (maybe, ate), (ate, steak), (steak, eye), (eye, would), (would, point), (point, direction), (direction, booker)]","(0, 0.9999824398067418)",0,0.999982
7872118,"[(must, post), (post, every), (every, medium), (medium, available), (available, great), (great, red), (red, pilling), (pilling, ammo), (ammo, point), (point, moderate), (moderate, lefty), (lefty, critical), (critical, thinking), (thinking, skill), (skill, kick), (kick, agaisnt)]","(2, 0.9999895989205586)",2,0.999990
8439541,"[(gafftastically, funny), (funny, keep), (keep, race)]","(4, 0.4692790064766138)",4,0.469279


In [139]:
result = sample_df.merge(subset_date2, left_index=True, right_index = True)

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [140]:
result_restricted = result[['best_label', 'best_label_prob', 'text_clean_space', 'date']]
result_restricted.head()

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,best_label,best_label_prob,text_clean_space,date
8368084,1,0.327084,SPARKLIN TATERS!!,2019-08-09
7622971,3,0.949114,*puts on Tshirt* [LOOKIT ME! IM A FREEDOM FIGHTER](https://magaimg.net/img/8p67.png),2019-08-11
8319493,0,0.999982,"vegans get the fuck outta here with that soy shit, yo maybe if you ate some steak your eyes would point in the same direction, booker",2019-08-11
7872118,2,0.999990,"We must all post this on every media available! Great red pilling ammo, at some point the moderate lefties critical thinking skills will kick in agaisnt their will.",2019-08-10
8439541,4,0.469279,So gafftastically funny. . Keep him in this race!,2019-08-11


In [141]:
topic_0 = result_restricted[result_restricted['best_label']==0]
topic_1 = result_restricted[result_restricted['best_label']==1]
topic_2 = result_restricted[result_restricted['best_label']==2]
topic_3 = result_restricted[result_restricted['best_label']==3]

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [151]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)
topic_3[(topic_3['date']== datetime.strptime('2019-08-10', '%Y-%m-%d').date())].sort_values('best_label_prob', ascending = False).head(20)

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-151-4f627565ae95>:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,best_label,best_label_prob,text_clean_space,date
7750595,3,1.0,"So, at this point, it is no longer just a brazen prison hit. It is basically now an ""influence demonstrator"" on how much they can get away with. Case will get dismissed and sealed to better prioritize resources, news story gets dries up, investigation halted to avoid re-traumatizing the victims. The entire chain of people involved shows how much reach and depth is possible at any institution. No ""insurance policy"" is secure, is the lesson here. Those of you that think we can get new information if we extradited Assange to the US, here is a warning.",2019-08-10
8288903,3,1.0,"Oh shit, I didn't even consider this. This would make sense in a way. His lawyer was talking about ""fears of him being murdered"" or whatever, making public statements that were spread by the media. Could have been setting up an excuse for his disappearance, aka going in to hiding. This shit is crazy. Should pay attention to staff at the facility he was in, and whether or not they die in ""accidents"" or ""suicides"" in the near future. In other words, tying up loose ends, getting rid of anyone who knows what really happened. Of course, this is just tinfoil hat stuff, but it's all entirely possible, and lately it seems more and more likely to be reality.",2019-08-10
7813492,3,1.0,"The problem with a suicide watch is it leaves the prisoner vulnerable to an attack from outside their cell. What they do is strip the prisoner naked and put them in something called a turtle or Ferguson suit. However, unless there is constant monitoring (and that is very difficult to get people to do constantly), someone can easily slip in and attack the prisoner. What they should have done to keep this guy alive and talking was put him in a straight jacket and then put him in a locked padded room. That way they could limit who had access to him. This was amateur hour on the part of the Feds.",2019-08-10
7948098,3,1.0,&gt;and for the record. I think it's fishy as fuck too. just pointing out the fact that the still frame people thought looked like a diver..... not true based off the video footage . and she apparently died later from heart problems.. not in the water..... Im not trying to say there wasnt foul play. just the diver photo has been proven wrong by video footage. I didnt believe it at first either . til I watched several videos &gt; &gt;[https://youtu.be/3MJkKv9LOCc?t=269](https://youtu.be/3MJkKv9LOCc?t=269),2019-08-10
8059838,3,1.0,"&gt; I suggest you look up Occam's Razor, You do know that it **isn't** a ""law of nature"" like gravity or anything... it's an **observation** and a **generalization.** - I suggest you look up the word ""gullible""... because it definitely describes you. --- &gt;Epstein is dead Well, he's certainly been declared ""officially dead""... which relative to legal matters amounts to the same thing. (They can now ""bury"" the cases.) - But I rather **HIGHLY doubt** that he actually IS dead. --- &gt;They already raided his properties and busted open his safes. Maybe some of them... and so they **claim.**",2019-08-10
7974959,3,1.0,"&gt;Let me know what prosecutorial power Senator Lindsey Graham has over the FBI and DOJ because both the director of the FBI and the AG serve at the pleasure of the President. That's an irrelevant non sequitur to any point I've made about Graham. One doesn't need any prosecutorial powers to know that in this case there can not be accountability without criminal prosecutions. Period. The fact that the NeverTrumper RINO Graham thinks there can be further supports what most already know, he's just another deep state rat who thinks elites are above the law,",2019-08-10
8217259,3,1.0,Anyone else remember the full on media assault to discredit the notion that there was a global child sex trafficking ring being ran by some of the most high profile individuals in the world? Fast forward to now when the man charged with running a global child sex trafficking r

In [152]:
topic_0[(topic_0['date']== datetime.strptime('2019-08-10', '%Y-%m-%d').date())].sort_values('best_label_prob', ascending = False).head(20)

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,best_label,best_label_prob,text_clean_space,date
8333568,0,1.0,"**FOR THE LOVE OF GOD GET THIS PATRIOT A COAT! THATS 206,736 COATS HANDED OUT!** We are 73.33% of our goal to coat the 281,944 great people of Newark, New Jersey. Let's make sure everyone gets a coat in the United States! For every Centipede a coat, for every coat a Centipede! This bot was created by devious1087, feedback always welcome. [Meme Reference!](https://youtu.be/OiYVVT1aQFM)",2019-08-10
8374616,0,1.0,"&gt; I honestly have realized its pointless to depend on politicians to protect the right to own a gun. Neither you nor I have the ability to write or pass legislation. Politicians are my only intermediary to not be made a felon for exercising my constitutional rights. Obviously Im going to express my concern on this issue. Further, youre not doing any favors by acting like people who **dont want to have their rights stripped** are acting like rednecks in the woods If you dont actually give a shit about protecting your rights fine, but get the hell out of the way for those of us that do.",2019-08-10
8429105,0,1.0,"&gt; Never let them tell you they just want socialism like Scandinavia ... their goal is communism. The goal of ALL socialists and socialism itself is communism. This fact is not even in dispute by those of small brain and lost soul who subscribe to the sick ideology. Socialism is commie lite, or a stepping stone on the way to the ultimate goal. No state, no government, just a peaceful world full of robots driven to work as hard as possible for as little as it takes to survive, having surrendered their individuality, their freedom of speech, and the weaponry with which to protect them.",2019-08-10
8324031,0,1.0,"After 20 years in the Navy and only knowing of one suicide directly, I always questioned this stat. The bottom line is that it's closer to one vet a day. Still way higher than the general population, but 22 a day would put veterans above post OP transgenders, which is ludicrous. They came to this number by using a dubious study. Then politicians jumped on it as a talking point and the military mental health professionals found a great bullet point to put on posters. https://taskandpurpose.com/truth-22-veteran-suicides-day",2019-08-10
7956107,0,1.0,"**FOR THE LOVE OF GOD GET THIS PATRIOT A BRICK! THAT'S 152334 BRICKS HANDED OUT!** We are at **21.4560461824025822808593472%** of our goal to **BUILD THE WALL** starting from Imperial Beach, CA to Brownsville, Texas! Let's make sure everyone gets a brick in the United States! For every Centipede a brick, for every brick a Centipede! At this rate, the wall will be **1675.07135210569396298755608 MILES WIDE** and **446.421492722534887906345998 FEET HIGH** by tomorrow! **DO YOUR PART!**",2019-08-10
8044349,0,1.0,"You hit the proverbial nail on the head. The unchecked emotionalism of our youth. This can only be changed by proper education. The real culprits are an educational system that endorses feeling and emotion over fact and, media that encourages the same. The children have been lied to and told the universe came from ""nothingness"" and will return to ""nothingness"", that they are ""nothing but advanced primates"" and truth is a ""social construct"". So, why should they care about right or wrong? Or anything but to themselves? Checking down on guns will only encourage different methods of mass homocide/assualt. The answer is America needs to return to believing the King James Bible, the one this Nation was principled upon.",2019-08-10
7784904,0,1.0,"Duckduckgo is not a proper search engine, they rely on bing and google for their results. Yippy works but I think this search engine is still incomplete. Now I use yandex when I'm looking for something that may have been redacted (and yes some videos and article are erased from search results on google, bing and duckduckgo) I don't really like Yandex (too much russian in my taste) but I can't find any other proper search engine that doesn't rely 

In [153]:
topic_1[(topic_1['date']== datetime.strptime('2019-08-10', '%Y-%m-%d').date())].sort_values('best_label_prob', ascending = False).head(20)

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,best_label,best_label_prob,text_clean_space,date
7982902,1,1.0,"This is unusual for Brighton, it's an expensive place to live, doesn't have projects, combination of tourist and commuter town. So how could this happen? The long video shows them taking their bikes on the train to Brighton. Where from? At 20:45 the cameraman says 'Come to London'. For your convenience here are links to YT. The long video shows all their faces clearly. Not the best way to get away with a crime, but then they're not going to be very upset with a stern warning anyway. They look about 16 so should have a long criminal career ahead of them. Long version, 24 minutes - https://www.youtube.com/watch?v=6ZsJEjobRj4 One minute highlights - https://www.youtube.com/watch?v=7Mg-OKVtsC8 https://www.youtube.com/watch?v=wxgF8dITR-o",2019-08-10
7765560,1,1.0,"She stayed with him through cancer, he leaves her after she is diagnosed with breast cancer and 2.5 weeks before her lumpectomy. Dude is a real class act. I was bummed when he got busted because I am a testicular cancer survivor and Livestrong was really cool to me (sent me a bunch of free bracelets and a couple books) without me even asking for anything except info. But Lance ended up being such a shitty person that the 1 high-profile mens cancer charity tanked because of his fuckery. I didnt care about the PEDs, I did care that he is a straight-up douchebag when he thinks nobodys looking.",2019-08-10
8331749,1,1.0,"It is a spiritual warfare going on. These people believe in the devil and worship him. Spiritcooking, sex trafficking, every form of evil in order to have more and more demonic power. These people are fucking crazy. God intervened during the damn elections and got Trump in office I think. Put my conspiracy prophecy porn hat on. I don't care. The dude has been a last hope to wake the people up to the ultimate redpill. That we all need to follow Jesus. Return to Him and His ways. There is a spiritual problem in America. We need God fearing leaders again. The sheperds guarding the sheep are bad. Church over.",2019-08-10
8040043,1,1.0,"In Greece there was this ridiculous dude that run a newspaper. Cops were sent to arrest him... Amd the motherfucker had escape tunnels in his mansion. He wasn't even super rich or anything!! Then a year later or whatever... On his way somewhere.. His Porchse Cayenne crushed and he died in a blaze of fire... Near the airport! Again.. This was just an idiot, relatively rich yet not insanely rich douche that was proven to have escape tunnels in his mansion. If this Greek idiot had escape plans.. Then filthy rich Epstein definitely had one too.",2019-08-10
8368340,1,1.0,"I'll bet even money that within the first six months the room-temperature-IQ drama queen is at Hah-vahd and realizes how truly *ignorant* he really is, he'll be the FIRST to complain of (insert this week's Victimhood Reason), the professors are: racist, homophobic, germophobic, ET-phone-homophobic, blah blah blah. Meanwhile, his **genuinely** BRILLIANT classmate Kyle Kashuv gets a *shiv between the shoulder-blades* (i.e., his acceptance to the same institution abruptly RESCINDED due to some *juvenile* comments he made on twitter - that 110% of his peers made FAR worse - years ago). No hypocrisy here. [Don't eat that!](http://magaimg.net/img/7dpc.jpg)",2019-08-10
8371362,1,1.0,"Those who do not frequent 8ch and half-chan have no idea of how diverse the anons are and how many of them are in positions of importance. Most people see them as img boards full of 14 year old hackers, anti semites and the depraved of society. A lot of anons just get quite bored with the usual nonsensical background noise of the internet and life in general and need the playpen of others with quick wits, keen observation skills, critical thinking skills and unusual intelligence to escape the mundane. The fact that it scares off normies is exactly why they will always thrive - usually under the radar.",2019-08-10
8411539,1,1.0,"I've

In [154]:
topic_2[(topic_2['date']== datetime.strptime('2019-08-10', '%Y-%m-%d').date())].sort_values('best_label_prob', ascending = False).head(20)

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,best_label,best_label_prob,text_clean_space,date
8399698,2,1.0,"We had a presence of Marxist professors in the 60s/70s who trained and indoctrinated the current wave of neo-Marxist professors and educators, as well as journalists. This combined with the amalgamation of Intersectionalism and feminism by black feminists (namely Kimberl Crenshaw) in the mid-80s, which is the bedrock center of most Fake News' reporting and decision making today. (We're now seeing interectionalism infiltrate corporations, too. Whether it will stick remains to be seen.) Marxists/neo-Marxists have used education institutions as brainwashing and recruiting facilities for over 60 years, and we're dealing with the byproduct now. The Fake News, as we know now it, didn't exist before then.",2019-08-10
8121480,2,1.0,"Every time someone with a family breaks the law and gets arrested, they are separated from their family. Why do people only fake outrage when this happens to illegal aliens? 1. Don't break the law and enter the US illegally. 1. If you continue to break the law, you run the risk of being arrested. 1. It's your duty and responsibility to have a plan for your children if that happens. If not, the state will step in and provide care for them. But it will take time, be needlessly stressful on your famiky, and you might not like where they end up. All of that is on you. Whether you are an illegal alien, a black, white, man, woman, or anything between. You go to jail, your kids aren't going with you.",2019-08-10
7666901,2,1.0,"I guarantee only a few of the protags would be decent people. The rest would be hill folk cliches spewing racism out every other word and the crowd would cheer when they died. And I wouldn't be surprised if the surviving heroes were secretly Democrats trying to hide their secret in the evil red states. Hollywood is stupid but they aren't going to make a movie about killing Republicans if they arent going to dehumanize at least some of them. If anything, the recent shootings are the real reason this got pulled. Not because it was obviously about killing conservatives for sport",2019-08-10
7713056,2,1.0,"People say that the Clinton body count means nothing simply because theyre famous and know a lot of people, therefor its natural to be linked to some of those peoples deaths. My counter is this.... I dont know of an Obama body count. If it was just some right wing smear, we would do it to Obama also right? Well we dont, because it doesnt exist even though the Obamas probably know just as many or more people than the clintons. This body count issue is exclusively a Clinton thing. No prominent democrat has a body count except the Clintons. Which takes away from the claims that this is just some right wing conspiracy.",2019-08-10
7984941,2,1.0,"From how high up in the DOJ and FBI did the **""ok""** come ? This is a case where the consequence for the ***""ugly-elite""*** is worse then revealing how powerful they really are. They did a FU, in your face murder so that they could continue sexually molesting children. God only knows how many, who have been smuggled across our Southern border are being kept in dog cages in basements or attics to be taken out and used when the ***""ugly-elite""*** has a sexual emergency - then God help the baby. &amp;#x200B; **We who fight to build a fence are fighting on the side of God !**",2019-08-10
7896268,2,1.0,"I argued with someone in WoW general chat today for 15 minutes. He was rapid firing anecdotes, lies, personal attacks, and even literally REEEEED at one point (actually spammed that) to drown out chat. I defeated every single point in a rapid fire motion, pushing them away and turning them on them on him as fast as they came. By the end I caught him in a lie, pinned it on him, and declared lying disgusting. He finally shut up. Did I cure his TDS? No. Hes a commie and hell always hate trump. Did I defeat TDS? Yes, and it felt great.",2019-08-10
8230004,2,1.0,"This whole trump body count thing

### LDA

In [64]:
import gensim
import gensim.corpora as corpora

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [70]:
id2word = corpora.Dictionary(subset_date2['tokens_lemma'])
print('Making corpus...')
corpus = [id2word.doc2bow(text) for text in subset_date2['tokens_lemma']]

lda = gensim.models.LdaMulticore(corpus = corpus, id2word = id2word,
                                     num_topics =4, workers = 3,
                                     passes=10, iterations=100, random_state=0, eval_every=None)

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Making corpus...


In [68]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [71]:
gensimvis.prepare(lda, corpus, id2word)

/Users/christineibaraki/Documents/workspace/RedditQuarantineNLP/env/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.002446  0.107431       1        1  30.289986
3     -0.051283  0.017448       2        1  29.907504
0     -0.089048 -0.078662       3        1  24.180933
1      0.137885 -0.046217       4        1  15.621577, topic_info=        Term        Freq        Total Category  logprob  loglift
180      guy  568.000000   568.000000  Default  30.0000  30.0000
44   epstein  762.000000   762.000000  Default  29.0000  29.0000
331  clinton  539.000000   539.000000  Default  28.0000  28.0000
357  suicide  561.000000   561.000000  Default  27.0000  27.0000
484    watch  399.000000   399.000000  Default  26.0000  26.0000
..       ...         ...          ...      ...      ...      ...
564    white  100.502759   520.902783   Topic4  -5.9503   0.2111
114     many   94.608984   431.320334   Topic4  -6.0108   0.3394
510      day   92.215894   389.709656   Topic4  -6.0364   0.4152
149     time   97.964567  1000.188805   Topic4  -5.9759  -0.4668
493       go   85.591343   703.190377   Topic4  -6.1109  -0.2495

[352 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
390       2  0.874127    12
390       3  0.130189    12
8263      2  0.975039  2008
1041      1  0.078075  2016
1041      2  0.827590  2016
...     ...       ...   ...
133       4  0.046951  year
1526      1  0.208301   yes
1526      2  0.636475   yes
1526      3  0.111865   yes
1526      4  0.046289   yes

[698 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 1, 2])